In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as scipy_img
from cv2 import VideoWriter, VideoWriter_fourcc

In [ ]:
# From: https://stackoverflow.com/a/42154921
def perlin(x: np.ndarray, y: np.ndarray, seed=0):
    # permutation table
    np.random.seed(seed)
    p = np.arange(256, dtype=int)
    np.random.shuffle(p)
    p = np.stack([p, p]).flatten()
    # coordinates of the top-left
    xi = x.astype(int)
    yi = y.astype(int)
    # internal coordinates
    xf = x - xi
    yf = y - yi
    # fade factors
    u = fade(xf)
    v = fade(yf)
    # noise components
    n00 = gradient(p[p[xi] + yi], xf, yf)
    n01 = gradient(p[p[xi] + yi + 1], xf, yf - 1)
    n11 = gradient(p[p[xi + 1] + yi + 1], xf - 1, yf - 1)
    n10 = gradient(p[p[xi + 1] + yi], xf - 1, yf)
    # combine noises
    x1 = lerp(n00, n10, u)
    x2 = lerp(n01, n11, u)  # FIX1: I was using n10 instead of n01
    return lerp(x1, x2, v)  # FIX2: I also had to reverse x1 and x2 here


def lerp(a, b, x):
    "linear interpolation"
    return a + x * (b - a)


def fade(t):
    "6t^5 - 15t^4 + 10t^3"
    return 6 * t ** 5 - 15 * t ** 4 + 10 * t ** 3


def gradient(h, x, y):
    "grad converts h to the right gradient vector and return the dot product with (x,y)"
    vectors = np.array([[0, 1], [0, -1], [1, 0], [-1, 0]])
    g = vectors[h % 4]
    return g[:, :, 0] * x + g[:, :, 1] * y

In [ ]:
width = 640
height = 480
FPS = 10
seconds = 15  # 1.0
frames = int(FPS * seconds)

# Could use "test.mp4" and the "H264" fourcc codec below.
FILENAME = "test.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(FPS), (width, height))

for frame_num in range(frames):
    # Start with a black background.
    frame = np.zeros((height, width, 3), dtype=np.uint8)

    lin_x = np.linspace(0, 5, width, endpoint=False)
    lin_y = np.linspace(0, 5, height, endpoint=False)
    x, y = np.meshgrid(lin_x, lin_y)
    noise = perlin(x, y, seed=frame_num)

    # Make noise between 0 and 1
    noise -= np.min(noise)
    noise /= np.max(noise)

    frame[:, :, 0] = noise * 255
    frame[:, :, 1] = noise * 255
    frame[:, :, 2] = noise * 255
    video.write(frame)

video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)